In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


In [ ]:
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# import seaborn as sns
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from catboost import CatBoostRegressor

In [3]:
df = pd.read_csv("/kaggle/input/playground-series-s4e12/train.csv")
# df.head()

In [ ]:
df_test = pd.read_csv("/kaggle/input/playground-series-s4e12/test.csv")
df_test.head()

In [ ]:
df.info()

In [ ]:
df.describe().round(1)

In [ ]:
df.isna().sum()

In [ ]:
sns.heatmap(df.isna().sort_values(by=["Occupation"]) == True)

In [4]:
# df.dropna(inplace = True)
for non_object_column in df.select_dtypes(exclude = "object").columns:
    df[non_object_column].fillna(df[non_object_column].mean(), inplace = True)

df.dropna(inplace = True)

In [5]:
df.shape

(775274, 21)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(df.select_dtypes(exclude = "object").corr(), annot = True, cmap = "Reds")

In [ ]:
df_categorical = df.select_dtypes('object')
for object_column in df_categorical.columns:
    print(object_column, {df[object_column].nunique()}, ': ', df[object_column].unique(), '\n')

In [6]:
df.drop(['id', 'Policy Start Date'], axis = 1, inplace = True)

In [ ]:
for non_object_column in df.select_dtypes(exclude = "object").columns:
    if non_object_column != 'Premium Amount':
        df[non_object_column] = df[non_object_column] / df[non_object_column].max()

In [ ]:
df.head()

In [ ]:
df["Policy Start Date"] = pd.to_datetime(df["Policy Start Date"].apply(lambda x: x[:10]))

In [ ]:
pd.get_dummies(df_categorical, drop_first = True, dtype = int)

In [7]:
df = pd.concat(
    [
        df.select_dtypes(exclude = "object"), 
        pd.get_dummies(df.select_dtypes('object'), drop_first = True, dtype = int)
    ],
    axis = "columns"
)
df.head()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount,Gender_Male,...,Policy Type_Comprehensive,Policy Type_Premium,Customer Feedback_Good,Customer Feedback_Poor,Smoking Status_Yes,Exercise Frequency_Monthly,Exercise Frequency_Rarely,Exercise Frequency_Weekly,Property Type_Condo,Property Type_House
0,19.0,10049.0,1.0,22.598761,2.0,17.0,372.00000,5.0,2869.0,0,...,0,1,0,1,0,0,0,1,0,1
2,23.0,25602.0,3.0,47.177549,1.0,14.0,592.92435,3.0,567.0,1,...,0,1,1,0,1,0,0,1,0,1
4,21.0,39651.0,1.0,20.376094,0.0,8.0,598.00000,4.0,2022.0,1,...,0,1,0,1,1,0,0,1,0,1
7,48.0,127237.0,2.0,5.769783,1.0,11.0,398.00000,5.0,111.0,0,...,1,0,0,0,0,0,1,0,1,0
9,44.0,52447.0,2.0,20.473718,1.0,9.0,635.00000,3.0,64.0,1,...,1,0,0,1,0,0,0,0,1,0


In [ ]:
df.dtypes

In [8]:
x = df.drop(['Premium Amount'], axis = 1)
y = df['Premium Amount']

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

In [10]:
# model1 = LinearRegression()
# model2 = RandomForestRegressor(n_estimators=28, random_state=0)
# model3 = PolynomialFeatures(degree=4)
model4 = CatBoostRegressor()

In [ ]:
x_train = model3.fit_transform(x_train)
x_test = model3.fit_transform(x_test)
x = model3.fit_transform(x)

In [11]:
# model = model3
# model.fit(x_train, y_train)

model = model4
model.fit(x_train, y_train)

Learning rate set to 0.113084
0:	learn: 861.2184375	total: 120ms	remaining: 1m 59s
1:	learn: 858.9230263	total: 170ms	remaining: 1m 24s
2:	learn: 856.8889407	total: 219ms	remaining: 1m 12s
3:	learn: 855.4669054	total: 269ms	remaining: 1m 6s
4:	learn: 853.9251226	total: 322ms	remaining: 1m 4s
5:	learn: 852.9623119	total: 368ms	remaining: 1m
6:	learn: 851.7893705	total: 415ms	remaining: 58.9s
7:	learn: 851.0076795	total: 460ms	remaining: 57.1s
8:	learn: 850.3946045	total: 506ms	remaining: 55.7s
9:	learn: 849.9079445	total: 553ms	remaining: 54.8s
10:	learn: 848.9945229	total: 605ms	remaining: 54.4s
11:	learn: 848.3908132	total: 651ms	remaining: 53.6s
12:	learn: 847.8584524	total: 700ms	remaining: 53.1s
13:	learn: 847.5728214	total: 746ms	remaining: 52.5s
14:	learn: 847.3824104	total: 794ms	remaining: 52.1s
15:	learn: 847.1260799	total: 841ms	remaining: 51.7s
16:	learn: 846.8842217	total: 887ms	remaining: 51.3s
17:	learn: 846.7995109	total: 929ms	remaining: 50.7s
18:	learn: 846.3283728	tot

In [12]:
y_pred = model.predict(x_test)

In [13]:
def rmsle(predictions, targets):
    """Calculate the root mean squared logarithmic error between predictions and targets"""
    return np.sqrt(np.mean((np.log(predictions + 1) - np.log(targets + 1)) ** 2))

In [14]:
rmsle(y_pred, y_test)

1.1361847833337448

In [15]:
model .fit(x, y)

Learning rate set to 0.117142
0:	learn: 861.9293186	total: 86.7ms	remaining: 1m 26s
1:	learn: 859.6203142	total: 147ms	remaining: 1m 13s
2:	learn: 857.7854342	total: 204ms	remaining: 1m 7s
3:	learn: 856.3223924	total: 260ms	remaining: 1m 4s
4:	learn: 854.7222335	total: 318ms	remaining: 1m 3s
5:	learn: 853.7335400	total: 374ms	remaining: 1m 1s
6:	learn: 852.3558048	total: 431ms	remaining: 1m 1s
7:	learn: 851.6488318	total: 486ms	remaining: 1m
8:	learn: 850.8629400	total: 544ms	remaining: 59.9s
9:	learn: 850.3521356	total: 603ms	remaining: 59.7s
10:	learn: 849.4888777	total: 672ms	remaining: 1m
11:	learn: 848.9600584	total: 730ms	remaining: 1m
12:	learn: 848.6577287	total: 787ms	remaining: 59.7s
13:	learn: 848.3916188	total: 842ms	remaining: 59.3s
14:	learn: 848.1048499	total: 901ms	remaining: 59.2s
15:	learn: 847.8450360	total: 958ms	remaining: 58.9s
16:	learn: 847.6243350	total: 1.01s	remaining: 58.7s
17:	learn: 847.5406819	total: 1.07s	remaining: 58.2s
18:	learn: 847.0647009	total: 1.

In [ ]:
x.shape

In [16]:
df_test = pd.read_csv("/kaggle/input/playground-series-s4e12/test.csv")

# for non_object_column in df_test.select_dtypes(exclude = "object").columns:
#     df_test[non_object_column] = df_test[non_object_column] / df_test[non_object_column].max()

for non_object_column in df_test.select_dtypes(exclude = "object").columns:
    df_test[non_object_column].fillna(df_test[non_object_column].mean(), inplace = True)

df_test["Policy Start Date"] = pd.to_datetime(df_test["Policy Start Date"].apply(lambda x: x[:10]))

df_test = pd.concat(
    [
        df_test.select_dtypes(exclude = "object"), 
        pd.get_dummies(df_test.select_dtypes('object'), drop_first = True, dtype = int)
    ],
    axis = "columns"
)

x_df_test = df_test.drop(['id', 'Policy Start Date'], axis = 1)

print(x_df_test.shape)
x_df_test.head()

(800000, 28)


,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Gender_Male,Marital Status_Married,...,Policy Type_Comprehensive,Policy Type_Premium,Customer Feedback_Good,Customer Feedback_Poor,Smoking Status_Yes,Exercise Frequency_Monthly,Exercise Frequency_Rarely,Exercise Frequency_Weekly,Property Type_Condo,Property Type_House
0,28.0,2310.0,4.0,7.657981,1.004873,19.0,592.904749,1.0,0,0,...,0,0,0,1,1,0,0,1,0,1
1,31.0,126031.0,2.0,13.381379,1.004873,14.0,372.000000,8.0,0,1,...,0,1,1,0,1,0,1,0,0,0
2,47.0,17092.0,0.0,24.354527,1.004873,16.0,819.000000,9.0,0,0,...,1,0,0,0,1,1,0,0,1,0
3,28.0,30424.0,3.0,5.136225,1.000000,3.0,770.000000,5.0,0,0,...,1,0,0,1,1,0,0,0,0,1
4,24.0,10863.0,2.0,11.844155,1.004873,14.0,755.000000,7.0,1,0,...,0,1,0,0,0,0,0,1,0,1


In [17]:
Y_pred_test = model.predict(x_df_test)

In [18]:
df_sub = pd.read_csv("/kaggle/input/playground-series-s4e12/sample_submission.csv")
df_sub.head()

,id,Premium Amount
0,1200000,1102.545
1,1200001,1102.545
2,1200002,1102.545
3,1200003,1102.545
4,1200004,1102.545


In [19]:
df_test['Premium Amount'] = Y_pred_test.round(3)
df_submission = df_test[['id', 'Premium Amount']]

print(df_submission.shape)
df_submission.head()

(800000, 2)


,id,Premium Amount
0,1200000,1590.688
1,1200001,1177.880
2,1200002,1073.606
3,1200003,1130.102
4,1200004,972.793


In [20]:
df_submission.to_csv("submissions.csv", index=False)

In [ ]:
df_test.isna().sum()